In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [21]:
import numpy as np
import pandas as pd
import time

from src.recommender import train_mf, reviews_dataset, evaluate_recommendations
from src.recommender.mf_recommender import update_Y, preprocess, get_reviews_by_user, update_X

In [3]:
reviews_train, reviews_validation, reviews_test, images = reviews_dataset()

In [4]:
recommender = train_mf(
    train_data=reviews_train,
    images=images,
    d=20,
    alpha=.1,
    beta=.1,
    max_als_epochs=10,
    sgd_learning_rate=.1,
    max_sgd_epochs=1000000,
    sgd_batch_size=1,
)

als_epoch=0
Computing X
151.95s
Computing Y
epoch=100000, elapsed: 22.97s, average: 22.97s, average_update_size=8.54e-04
epoch=200000, elapsed: 46.66s, average: 23.33s, average_update_size=1.03e-03
epoch=300000, elapsed: 69.96s, average: 23.32s, average_update_size=1.08e-03
epoch=400000, elapsed: 90.71s, average: 22.68s, average_update_size=1.15e-03
epoch=500000, elapsed: 109.61s, average: 21.92s, average_update_size=1.17e-03
epoch=600000, elapsed: 129.20s, average: 21.53s, average_update_size=1.28e-03
epoch=700000, elapsed: 148.83s, average: 21.26s, average_update_size=1.32e-03
epoch=800000, elapsed: 168.41s, average: 21.05s, average_update_size=1.34e-03
epoch=900000, elapsed: 188.22s, average: 20.91s, average_update_size=1.39e-03
epoch=1000000, elapsed: 207.90s, average: 20.79s, average_update_size=1.41e-03
207.90s
Computing loss
loss=2.6640424009958927
66.72s
als_epoch=1
Computing X
133.39s
Computing Y
epoch=100000, elapsed: 19.13s, average: 19.13s, average_update_size=5.70e-04
epoc

epoch=500000, elapsed: 97.03s, average: 19.41s, average_update_size=1.20e-04
epoch=600000, elapsed: 116.39s, average: 19.40s, average_update_size=1.16e-04
epoch=700000, elapsed: 135.83s, average: 19.40s, average_update_size=1.26e-04
epoch=800000, elapsed: 155.26s, average: 19.41s, average_update_size=1.19e-04
epoch=900000, elapsed: 174.64s, average: 19.40s, average_update_size=1.38e-04
epoch=1000000, elapsed: 193.99s, average: 19.40s, average_update_size=1.22e-04
193.99s
Computing loss
loss=0.8978554996878474
53.36s


In [15]:
Y_copy = recommender.Y.copy()

In [22]:
d=20
alpha=.1
beta=.1
train_data, I, tags, n, m, k = preprocess(reviews_train, images)

image_indices, ratings, start_indices, end_indices = get_reviews_by_user(train_data)

X = np.random.normal(size=(n, d))
Y = recommender.Y.copy()

print("Computing X")
start = time.time()
update_X(
    X=X,
    Y=Y,
    I=I,
    image_indices=image_indices,
    ratings=ratings,
    start_indices=start_indices,
    end_indices=end_indices,
    n=n,
    d=d,
    alpha=alpha,
)
end = time.time()
print(f"{end - start:.2f}s")

Computing X
116.92s


In [37]:
sgd_learning_rate=.1
max_sgd_epochs=100000
sgd_batch_size=10

Y = recommender.Y.copy()

update_Y(
    X=X,
    Y=Y,
    I=I,
    image_indices=image_indices,
    ratings=ratings,
    start_indices=start_indices,
    end_indices=end_indices,
    n=n,
    k=k,
    d=d,
    beta=beta,
    learning_rate=sgd_learning_rate,
    max_epochs=max_sgd_epochs,
    batch_size=sgd_batch_size,
)

epoch=100000, elapsed: 117.93s, average: 117.93s, delta_Y_size=2.12e+02, Y_size=3.55e+03, percent_change=5.98e-02


In [5]:
reviews_validation_by_user = reviews_validation.groupby("user_index").apply(lambda group: (group["image_index"].to_numpy(), group["rating"].to_numpy()))

KeyError: 'user_index'

In [ ]:
validation_loss = 1/n * sum(1 / len(image_ids) * sum((ratings - image_tags[image_ids] @ (Y @ X[u])) ** 2) for u, (image_ids, ratings) in reviews_validation_by_user.items() if len(image_ids) > 0)
validation_loss

In [ ]:
recommender = train_knn(reviews_train, word_embedding)

In [ ]:
evaluate_recommendations(recommender, reviews_validation)